<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/RecercaExtra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import torch
import cv2
import csv
import random
import numpy as np
import ast
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl
import json

from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io
from google.colab import drive

# ORIGEN = '/content/drive/MyDrive/HAM10000/skin-cancer-mnist-ham10000/'

drive.mount('/content/drive')


PATH_TRAINING_DISTANCES = "/content/drive/MyDrive/Features/Embeddings/embeddings_pesos_resnet152multiclass_3.pt.txt" #embeddings training
DISTANCES_IMAGES = "/content/drive/MyDrive/Features/Dist/1/distances.txt" #distàncies de totes les imatges de test
IMATGES_INTERES = ["ISIC_0027790"] #imatges amb els patrons que cercam

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# !mkdir -p ~/.kaggle
# !mv /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [149]:
dataframe = pd.read_csv(DISTANCES_IMAGES, sep=';')
embeddings_training = pd.read_csv(PATH_TRAINING_DISTANCES, sep=',')
files_interes = []

for i in IMATGES_INTERES:
    fila = dataframe[dataframe['test_img'] == i]
    files_interes.append(fila)

for i in files_interes:
  vector_features = [[],[]]

  imatge_test = i['test_img'].iloc[0]
  imatges_training = ast.literal_eval(i['top_images_names'].iloc[0])
  imatges_training_jpg = [img + ".jpg" for img in imatges_training]

  training = embeddings_training[embeddings_training['Image Name'].isin(imatges_training_jpg)][['FC Input','Image Name']]
  distances_training = training['FC Input'].to_list()
  vector_features[0].append(training['Image Name'].to_list())

  for j in distances_training:
    vector_features[1].append(ast.literal_eval(j)) #distàncies de les imatges d'entrenament guardades, ens interessaria els noms també

  #falta cercar el vector de distàncies de la imatge de test i fer les comparacions adients.



In [150]:
print(vector_features)

[[['ISIC_0024368.jpg', 'ISIC_0031527.jpg', 'ISIC_0028275.jpg', 'ISIC_0031124.jpg', 'ISIC_0033502.jpg']], [[0.7856606841087341, 0.5121546387672424, 0.8682350516319275, 0.5226900577545166, 0.49093466997146606, 0.5905008316040039, 0.5312695503234863, 0.5049769282341003, 0.5183864235877991, 0.4926767349243164, 0.7994023561477661, 0.4880546033382416, 0.5394668579101562, 0.5927191376686096, 0.5383214354515076, 0.35443344712257385, 0.20803594589233398, 0.3532955050468445, 0.5134688019752502, 0.3074725866317749, 0.49181753396987915, 0.5385457277297974, 0.6005908846855164, 0.4601752460002899, 0.7577992677688599, 0.49745941162109375, 0.5413554310798645, 0.5456147193908691, 0.53293377161026, 0.46779149770736694, 0.5284411311149597, 0.31782105565071106, 0.2660854756832123, 0.6481049656867981, 0.3725912868976593, 0.5743991732597351, 0.3776357173919678, 0.3450734317302704, 0.2892196774482727, 0.6056393384933472, 0.4339463412761688, 0.5147116780281067, 0.6768426299095154, 0.4980008900165558, 0.530834